In [50]:
import json
import os
from types import SimpleNamespace

"""
author(s): Arnold Unterauer
"""

####################

env_name = "3s_vs_3z"
compare_token_0 = "qtran__2022-07-17_19-20-46_276"
# compare_token_1 = "qtran__2022-07-16_11-35-04_83"

####################

def extract_data(env_name, unique_token):
    save_eval_buffer_path = os.path.join(os.getcwd(), "results", "models", "custom", env_name, unique_token)
    with open(os.path.join(save_eval_buffer_path, "unique_eval_buffer.txt"), "r") as f:
        eval_buffer = json.load(f)

    with open(os.path.join(save_eval_buffer_path, "args.txt"), "r") as f:
        args = json.load(f)
    args = SimpleNamespace(**args)

    rewards_mac = eval_buffer[0][0]
    actions_mac = eval_buffer[0][1]
    if args.train_hidden_a_learner or args.train_simultaneously or args.pure_antagonist or args.only_gan:
        rewards_hidden_mac = eval_buffer[1][0]
        actions_hidden_mac = eval_buffer[1][1]
        return True, args.n_antagonists, rewards_mac, actions_mac, rewards_hidden_mac, actions_hidden_mac
    return False, 0, rewards_mac, actions_mac, None, None

def get_bined_actions(rewards, actions):
    unique_rewards = [sum(reward) / len(reward) for reward in rewards]
    rewards_set = list(set(unique_rewards))
    sorted_rewards_set = sorted((e,i) for i,e in enumerate(rewards_set))

    index_reward_dict = {}
    for e in sorted_rewards_set:
        index_reward_dict['{}'.format(e[0])] = e[1]

    bined_rewards_indices = [[] for _ in range(len(rewards_set))]
    for i, r in enumerate(unique_rewards):
        bined_rewards_indices[index_reward_dict['{}'.format(r)]].append(i)
    bined_actions = [[actions[r] for r in bined_rewards] for bined_rewards in bined_rewards_indices]

    return bined_actions

####################

hidden_exist_0, n_antagonists, \
rewards_mac_0, actions_mac_0, \
rewards_hidden_mac_0, actions_hidden_mac_0 \
    = extract_data(env_name, compare_token_0)

bined_actions_mac = get_bined_actions(rewards_mac_0, actions_mac_0)
if hidden_exist_0:
    bined_actions_hidden_mac = get_bined_actions(rewards_hidden_mac_0, actions_hidden_mac_0)

actions = []
for bined_action_mac in bined_actions_mac:
    bin_actions = []
    for action in bined_action_mac:
        for a in action[- n_antagonists:]:
            bin_actions.append(a)
    actions.append(bin_actions)

hidden_actions = []
for bined_action_hiiden_mac in bined_actions_hidden_mac:
    bin_actions = []
    for action in bined_action_hiiden_mac:
        for a in action[- n_antagonists:]:
            bin_actions.append(a)
    hidden_actions.append(bin_actions)

same_actions = []
for i, action in enumerate(actions):
    same_action = []
    for j in range(len(action)):
        if actions[i][j] == hidden_actions[i][j]:
            same_action.append(True)
        else:
            same_action.append(False)
    same_actions.append(same_action)

unique_rewards = [sum(reward) / len(reward) for reward in rewards_mac_0]
rewards_set = list(set(unique_rewards))
sorted_rewards_set = sorted((e,i) for i,e in enumerate(rewards_set))

same_portion = []
for a_i in range(len(same_actions)):
    same_portion.append((sorted_rewards_set[a_i][0], sum(same_actions[a_i]) / len(same_actions[a_i])))

print("(reward, portion_same)")
print(same_portion)

# TODO rewards are wrong

(reward, portion_same)
[(0.0, 0.5719784449576597), (0.003676470588235294, 0.6104651162790697), (0.007352941176470588, 0.07792207792207792), (0.011029411764705883, 0.14814814814814814), (0.022058823529411766, 0.3146853146853147), (0.025735294117647058, 0.7777777777777778), (0.029411764705882353, 0.24), (0.05514705882352941, 0.0), (0.058823529411764705, 1.0), (0.29411764705882354, 1.0), (0.3235294117647059, 1.0), (0.34558823529411764, 1.0), (0.35294117647058826, 1.0), (0.35661764705882354, 1.0), (0.3602941176470588, 1.0), (0.3639705882352941, 0.6666666666666666), (0.375, 1.0), (0.38235294117647056, 0.5), (0.7058823529411765, 1.0), (0.7242647058823529, 1.0), (0.7352941176470589, 0.9534883720930233), (0.7647058823529411, 1.0)]
